In [ ]:
]activate ..

In [ ]:
include("main.jl")

In [ ]:
using .AudioInference

In [ ]:
trr = tones_with_noise(10,); nothing

In [ ]:

#   tr, _ = AudioInference.generate(AudioInference.generate_scene, AudioInference.args)
#   duration, _, sr, = AudioInference.get_args(tr)
#   AudioInference.plot_gtg(underlying_gram(tr), duration, sr, 0, 100)


In [ ]:
sum(  underlying_gram(tr)[:, 147])

In [ ]:
(initial_tr, weight) = generate_initial_tr(trr); nothing

In [ ]:
tr = do_generic_inference(initial_tr, 40, (_...) -> nothing)

In [ ]:
underlying_gram(tr) = AudioInference.get_retval(tr)[1]
observed_gram(tr) = tr[:kernel => :scene]
error_gram(tr) = observed_gram(tr) - underlying_gram(tr)

In [ ]:
function get_thresholded(img, threshold)
  if threshold < 0
    return img .< threshold
  else
    return img .> threshold
  end
end

In [ ]:
area(((x1, y1), (x2, y2))) = (y2-y1) * (x2-x1)

In [ ]:
duration, _, sr, = AudioInference.get_args(tr)
AudioInference.plot_gtg(get_thresholded(underlying_gram(trr), 25), duration, sr, 0, 1)

In [ ]:
AudioInference.plot_gtg(error_gram(tr), duration, sr, 0, 100)
# AudioInference.plot_gtg(get_thresholded(sum(error_gram(tr), dims=1), 500), duration, sr, 0, 1)

In [ ]:
duration, sr

In [ ]:
normalize(img) = 1/maximum(img) * img
rectprod(img) = sum(img, dims=2)*sum(img, dims=1)

In [ ]:
MAX_TONE_VOL = 43.
TONE_COL_AVG_SUM = 600.
TONE_COL_SUM_RANGE = (500., 700.)

In [ ]:
using DSP

In [ ]:
function get_changepoints(strip; threshold_frac=0.2)
  # TODO: I could possibly be smarter by not doing abs, and looking at start vs end of regions
  cnvd = abs.(conv(strip/maximum(strip), [-2 -1 0 1 2]))
  changepts = []
  threshold = maximum(cnvd) * threshold_frac
  previdx = 0
  for (i, val) in enumerate(cnvd)
    if val > threshold
      if previdx < i - 1
        push!(changepts, i)
      end
      previdx = i
    end
  end
  return changepts
end

In [ ]:
sm = sum(error_gram(tr), dims=1)
cnvd = (conv(sm/maximum(sm), [-2 -1 0 1 2]))
AudioInference.plot_gtg(cnvd, duration, sr, -5, 5)

In [ ]:
maxval = maximum(cnvd)
findall(cnvd .> maxval * 0.2)

In [ ]:
changepts = []
threshold = maxval * 0.2
previdx = 0
for (i, val) in enumerate(cnvd)
  if val > threshold
    if previdx < i - 1
      push!(changepts, i)
    end
    previdx = i
  end
end

In [ ]:
changepoints = get_changepoints(error_gram(tr))

In [ ]:
st, nd = changepoints[1:2]

In [ ]:
col = normalize(sum(@view(error_gram(tr)[:, changepoints[2]:changepoints[3]]), dims=2))
AudioInference.plot_gtg(col, duration, sr, -1, 1)

In [ ]:
AudioInference.plot_gtg(get_thresholded(col, 0.1), duration, sr, 0, 1)

In [ ]:
conv(get_thresholded(col, 0.1), [1,0,-1])

In [ ]:
function get_regions(col)
  regions = []
  start = nothing
  last_was_true = false
  for i=1:length(col)
    if col[i]>0
      if !last_was_true
        start = i
      end
      last_was_true = true
    else
      if last_was_true
        push!(regions, (start, i))
        start = nothing
      end
      last_was_true = false
    end
  end
  return regions
end

In [ ]:
get_regions(get_thresholded(col, 0.1))

In [ ]:
struct Tones
  regions::Vector{Tuple{Int, Int}}
end
function analyze_col(col)
  thresholded = get_thresholded(col, 0.1)
  regions = get_regions(col)
  is_noise = sum(((st,nd),) -> nd-st, regions) > length(thresholded)/*0.7
  if is_noise
    analyze_noise(col)
  else
    analyze_tones(col, regions)
  end
end
function analyze_noise(col)
  return (:noise, sum(col)/length(col))
end
function analyze_tones(col, regions)
  
  return (:tones, [(x+y)/2 for (x,y) in regions])
end

In [ ]:
STRIP_THRESHOLD = 0.1
REGION_WIDTH = 5
HRW = Int(floor(REGION_WIDTH/2))

function get_changepoints(img)
  vals = normalize(sum(img, dims=1))
  filter = [-2 -1 0 1 2]
  convd = [filter*(@view vals[x:x+4]) for x=1:length(vals)-5]
  return findall(x -> (x[1]) > 0.3, convd)
end

# function get_vertical_bars(img)
#   vals = sum(img, dims=1)
#   maxval = maximum(vals)
#   threshold = STRIP_THRESHOLD * maxval
#   strips = []
# #   start = 1
# #   for x=2:length(vals)
# #     if (vals[x]-vals[x-1]) > 
# #   end
  
  
  
 
  
#   stripsum = sum(vals[1:REGION_WIDTH])
#   start = 1
#   for x=(HRW + 2):length(vals)-HRW
#     stripsum += sum(vals[x+HRW])
#     stripmean = stripsum/(x-start + 1)
#     region_sum = sum(vals[x-HRW:x+HRW])
    
#     if abs(region_sum/REGION_WIDTH - stripmean) > threshold
#       push!(strips, (2*start/198, 2*x/198))
#       start = x
#       stripsum = sum(vals[x])
#     end
#   end
#   return strips
# end

In [ ]:
get_changepoints(underlying_gram(tr))

In [ ]:
58 * 2/198

In [ ]:
using ImageMorphology

In [ ]:
boxes = component_boxes(label_components(get_thresholded(get_error_gram(tr), 0.6)))

In [ ]:
findall(x -> x > 100, area.(boxes))

In [ ]:
AudioInference.plot_gtg(AudioInference.get_retval(tr)[1])

In [ ]:
function draw_rect!(img, ((x1, y1), (x2, y2)); color=100)
  for x=x1:x2
    img[x, y1] = color
    img[x, y2] = color
  end
  for y=y1:y2
    img[x1, y] = color
    img[x2, y] = color
  end
end

In [ ]:
function vis_wav_with_rectangles(tr, rects)
  duration, _, sr, = AudioInference.get_args(tr)
  gram, scene_wave, = AudioInference.get_retval(tr)
  gram = copy(gram)
  for rect in rects
    draw_rect!(gram, rect)
  end
  AudioInference.plot_gtg(gram, duration, sr, 0, 100)
end

In [ ]:
vis_wav_with_rectangles(tr, [box for box in boxes if area(box) > 50])

In [ ]:
duration, _, sr, = AudioInference.get_args(tr)
AudioInference.plot_gtg(get_thresholded(get_error_gram(tr), 10), duration, sr, 0, 1)

We could do scans of horizontal strips the width of a tone up and down the image, checking which ones seem to get some good overlap.  These are frequencies at which we need to add/delete tones.  We can then look at bounding rectangles within the sub-image represented by that strip to help figure out the right start/endtime.  We could also just average the vertical part in that rectangle, and then use some 1d algorithm.

Likewise, we can do scans of vertical strips left&right to find where to add/delete noise.  In this direction, we don't have fixed widths (since we don't know the noise duration).  For this, we could first scan with a 1-pixel strip, saving for each pixel the total overlap or just 1/0 depending whether it seems like a noise pixel.  We can then just look at connected components of the resulting horizontal strip.

In [ ]:
AudioInference.plot_gtg(get_thresholded(underlying_gram(tr), 10), duration, sr, 0, 1)

In [ ]:
using .AudioInference: AudioSource, choicemap, generate_scene, plot_gtg



In [ ]:
function get_noise_sum(tr, spot=Int(198/2))
  sum(underlying_gram(tr)[:, spot])
end

In [ ]:
constraints = choicemap(
  (:kernel => :n_tones, 1),
  (:world => :waves => AudioSource(1) => :is_noise, true),
  (:world => :waves => AudioSource(1) => :amp, 25.),
  (:world => :waves => AudioSource(1) => :duration, 0.5),
  (:world => :waves => AudioSource(1) => :onset, 0.75)
)
t, _ = AudioInference.generate(AudioInference.generate_scene, AudioInference.args, constraints)
duration, _, sr, = AudioInference.get_args(t)


In [ ]:
get_constraints(amp) = choicemap(
  (:kernel => :n_tones, 1),
  (:world => :waves => AudioSource(1) => :is_noise, true),
  (:world => :waves => AudioSource(1) => :amp, amp),
  (:world => :waves => AudioSource(1) => :duration, 0.5),
  (:world => :waves => AudioSource(1) => :onset, 0.75)
)
get_tr(amp) = AudioInference.generate(AudioInference.generate_scene, AudioInference.args, get_constraints(amp))[1]
amps = 2:100
sums = map(get_noise_sum ∘ get_tr, amps)

In [ ]:
AudioInference.plot(amps, sums)

In [ ]:
AudioInference.plot_gtg(underlying_gram(t), duration, sr, 0, 100)

In [ ]:
get_tone_constraints(erb) = choicemap(
  (:kernel => :n_tones, 1),
  (:world => :waves => AudioSource(1) => :is_noise, false),
  (:world => :waves => AudioSource(1) => :erb, erb),
  (:world => :waves => AudioSource(1) => :duration, 0.5),
  (:world => :waves => AudioSource(1) => :onset, 0.75)
)
get_tone_tr(erb) = AudioInference.generate(AudioInference.generate_scene, AudioInference.args, get_tone_constraints(erb))[1]
amps = 0.4:0.5:37.0

In [ ]:
AudioInference.plot_gtg(underlying_gram(get_tone_tr(12)), duration, sr, 0, 100)

In [ ]:
sum(underlying_gram(get_tone_tr(12))[1:64, 100])

In [ ]:
# get audiograms from traces
underlying_gram(tr) = AudioInference.get_retval(tr)[1]
observed_gram(tr) = tr[:kernel => :scene]
error_gram(tr) = observed_gram(tr) - underlying_gram(tr)

# threshold an image by the given value as a fraction of the max val in the image
function get_thresholded(img, threshold)
    threshold = threshold * maximum(img)
    if threshold < 0
        return img .< threshold
    else
        return img .> threshold
    end
end

# normalize an image so the max value is 1 and min is 0
normalize(img) = 1/(maximum(img) - minimum(img)) * img

# given a strip (row vector), estimates where there are "changepoints" from
# a region of one intensity to another
function get_changepoints(strip; threshold_frac=0.2)
    # TODO: I could possibly be smarter by not doing abs, and looking at start vs end of regions
    cnvd = abs.(conv(strip/maximum(strip), [-2 -1 0 1 2]))
    changepts = []
    threshold = maximum(cnvd) * threshold_frac
    previdx = 0
    for (i, val) in enumerate(cnvd)
        if val > threshold
            if previdx < i - 1
                push!(changepts, i)
            end
            previdx = i
        end
    end
    return changepts
end

# given a column bitvector, returns the connected regions
# format: [(a, b), (c, d), (e, f), ...]
function get_regions(col)
    regions = []
    start = nothing
    last_was_true = false
    for i=1:length(col)
        if col[i] > 0
            if !last_was_true
                start = i
            end
            last_was_true = true
        else
            if last_was_true
                push!(regions, (start, i-1))
                start = nothing
            end
            last_was_true = false
        end
    end
    if last_was_true
      push!(regions, (start, length(col)))
    end
    return regions
end

# given a column from one horizontal region of the soundwave,
# this approximately determines whether it's a noise or a tone,
# and the volume if it is a noise / freq if it is a tone
function analyze_col(col, width)
    thresholded = get_thresholded(col, 0.1)
    regions = get_regions(thresholded)
    is_noise = length(regions) > 0 && sum(nd-st for (st, nd) in regions) > length(thresholded)/0.7
    if is_noise
        analyze_noise(col, width)
    else
        analyze_tones(col, regions)
    end
end

# I derived this formula by analyzing the generated
# noises at different amp values:
noise_amp_for_col(col) = 1054 + 64*sum(col)

function analyze_noise(col, width)
    return (:noise, noise_amp_for_col(col/width))
end
function analyze_tones(col, regions)
    # TODO: how to interpret tone values as erbs?
    return (:tones, [(x+y)/2 for (x,y) in regions])
end

### get_analyzed_img_sections ###
function get_analyzed_audiogram_sections(gram)
    changepoints = get_changepoints(normalize(sum(gram, dims=1)))
    noises = []
    tones = []
    for i=1:length(changepoints)-1
        region = (changepoints[i], changepoints[i+1])
        col = sum(@view(gram[:, region[1]:region[2]]), dims=2)
        (typ, params) = analyze_col(col, region[2] - region[1])
        if typ === :noise
            push!(noises, (region, params))
        else
            for mean in params
                push!(tones, (region, mean))
            end
        end
    end
    return (noises, tones)
end
function analyze_underlying_gram(tr)
    err = underlying_gram(tr)
    pos = map(x -> x > 0 ? x : 0, err)
    # neg = map(x -> x < 0 ? x : 0, err)
    (posnoises, postones) = get_analyzed_audiogram_sections(pos)
    # (negnoises, negtones) = get_analyzed_audiogram_sections(neg)

    return (posnoises, postones)
end

In [ ]:
vals = analyze_underlying_gram(get_tone_tr(10))

In [ ]:
AudioInference.plot(0:0.1:37, vals)

In [ ]:
vals[(Int(15.3 * 10)) + 1]